In [ ]:
# accurate

import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
import joblib
import tkinter as tk
from tkinter import filedialog

def rgb_to_hsi(img):
    # Your rgb_to_hsi function code here

def extract_features(image):
    # Your extract_features function code here

# Load the trained K-NN model and scaler
knn_model = joblib.load('knn_model.pkl')
scaler = joblib.load('scaler.pkl')

# Function to classify an image
def classify_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Extract features
    features = extract_features(image)
    
    if features is not None:
        # Scale the features
        scaled_features = scaler.transform([features])
        
        # Predict the grade
        grade = knn_model.predict(scaled_features)
        
        return grade[0]
    else:
        return "Error: Unable to extract features from the image."

# Function to open file explorer for picking an image
def open_file_explorer():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Open file explorer and return the selected file path
    file_path = filedialog.askopenfilename()

    if file_path:
        # If a file is selected, classify the image
        predicted_grade = classify_image(file_path)
        print("Predicted Grade:", predicted_grade)
    else:
        print("No file selected.")

# Call the function to open file explorer
open_file_explorer()


In [8]:
#for paper

#zoom gA

import cv2
import os
import numpy as np


# Function to crop zoom the image within the bounding box and resize to a uniform size
def crop_zoom(image, x, y, w, h, zoom_factor=1.2, output_size=(800, 800)):
    # Calculate the zoomed area
    new_w = int(w / zoom_factor)
    new_h = int(h / zoom_factor)
    
    # Ensure the coordinates are within image boundaries
    start_x = max(x + (w - new_w) // 2, 0)
    start_y = max(y + (h - new_h) // 2, 0)
    end_x = min(start_x + new_w, image.shape[1])
    end_y = min(start_y + new_h, image.shape[0])
    
    # Crop the image to the zoomed area
    cropped_image = image[start_y:end_y, start_x:end_x]

    output_file_path = os.path.join(target_directory, "cropped.jpg")
    cv2.imwrite(output_file_path, cropped_image)
    
    # Resize the cropped image to the desired output size
    resized_image = cv2.resize(cropped_image, output_size)
    
    return resized_image

# Function to detect objects and get bounding box
def detect_and_draw_boxes(frame):
    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    output_file_path = os.path.join(target_directory, "gray.jpg")
    cv2.imwrite(output_file_path, gray)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (21, 21), 0)

    output_file_path = os.path.join(target_directory, "blurred.jpg")
    cv2.imwrite(output_file_path, blurred)
    
    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    output_file_path = os.path.join(target_directory, "thresholded.jpg")
    cv2.imwrite(output_file_path, thresholded)
    
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Find the contour with the largest area
        main_contour = max(contours, key=cv2.contourArea)

        # output_file_path = os.path.join(target_directory, "contour.jpg")
        # cv2.imwrite(output_file_path, main_contour)
        
        # Get bounding box coordinates around the main contour
        x, y, w, h = cv2.boundingRect(main_contour)
        return x, y, w, h, frame
    else:
        return None, None, None, None, frame

# Source directory containing the images
source_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr"

# Target directory for cropped zoom images
target_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output"

# Zoom factor and output size
zoom_factor = 1.2
output_size = (800, 800)

# Create the target directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Iterate over images in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        # Load the image
        input_file_path = os.path.join(source_directory, filename)
        image = cv2.imread(input_file_path)
        
        # Detect objects and get bounding box coordinates
        x, y, w, h, detected_image = detect_and_draw_boxes(image.copy())
        
        if x is not None and y is not None and w is not None and h is not None:
            # Zoom in within the bounding box and resize to uniform size
            zoomed_resized_image = crop_zoom(detected_image, x, y, w, h, zoom_factor, output_size)
            zoomed_output_file_path = os.path.join(target_directory, "grade_A_" + os.path.splitext(filename)[0] + "_zoom.jpg")
            cv2.imwrite(zoomed_output_file_path, zoomed_resized_image)

print("All cropped zoom images within bounding boxes saved to:", target_directory)


All cropped zoom images within bounding boxes saved to: C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output


In [14]:
#working resize turn bg tro black
import cv2
import os
import numpy as np

# Function to load, resize, and remove background from an image
def load_resize_remove_background(file_path, target_size):
    # Load the image
    image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        return None
    
    # Resize the image to target size
    resized_image = cv2.resize(image, (target_size, target_size))
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform adaptive thresholding to separate foreground from background
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert the thresholded image
    thresholded = cv2.bitwise_not(thresholded)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to store the foreground pixels
    mask = np.zeros_like(gray)

    # Draw contours on the mask
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)

    # Bitwise AND operation to extract foreground
    foreground = cv2.bitwise_and(resized_image, resized_image, mask=mask)

    # Set the background to black
    background_mask = (mask == 0)
    foreground[background_mask] = [0, 0, 0]
    
    return foreground

# Source directory containing the images
source_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output\op1"  # Replace with the actual path to your source image directory

# Target directory for resized images
target_directory = r"C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output\op2"  # Replace with the desired output directory for resized images

# Target size for resizing (square)
target_size = 300

# Create the target directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Iterate over images in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):  # Look for JPEG files
        # Load, resize, and remove background from the image
        input_file_path = os.path.join(source_directory, filename)
        processed_image = load_resize_remove_background(input_file_path, target_size)
        
        if processed_image is not None:
            # Save the original size processed image
            original_output_file_path = os.path.join(target_directory, os.path.splitext(filename)[0] + "_processed.png")
            cv2.imwrite(original_output_file_path, processed_image)

print("All images processed and saved to:", target_directory)


All images processed and saved to: C:\Users\Asus\OneDrive\Documents\School stuff\y 23-24\2nd sem\des2\Tuna_GRADES-20230522T082341Z-001\Tuna_GRADES\4ppr output\op2
